<a href="https://colab.research.google.com/github/umaimafawad711-code/hackathon/blob/main/Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# ---------------- Step 0: Imports ----------------
import pandas as pd
from datetime import datetime, timedelta
import os


In [15]:
# ---------------- Step 1: Load Medicine Data ----------------
CSV_PATH = "Medicine.csv"

# Create a sample medicine CSV if it doesn't exist (with your actual medicine list)
if not os.path.exists(CSV_PATH):
    medicine_list = [
        "Avastin 400mg Injection", "Augmentin 625 Duo Tablet", "Azithral 500 Tablet",
        "Ascoril LS Syrup", "Aciloc 150 Tablet", "Allegra 120mg Tablet", "Avil 25 Tablet",
        "Aricep 5 Tablet", "Amoxyclav 625 Tablet", "Atarax 25mg Tablet", "Azee 500 Tablet",
        "Anovate Cream", "Allegra-M Tablet", "Ascoril D Plus Syrup Sugar Free", "Alex Syrup",
        "Armotraz Tablet", "Augmentin Duo Oral Suspension", "Albendazole 400mg Tablet",
        "Arkamin Tablet", "Allegra 180mg Tablet", "Altraday Capsule SR", "Atarax 10mg Tablet",
        "Aldigesic-SP Tablet", "Aldactone Tablet", "Aricep 10 Tablet", "Aricep-M Tablet",
        "Andre I-Kul Eye Drop", "Anafortan 25 mg/300 mg Tablet", "Atarax Syrup", "Ambrodil-S Syrup"
    ]
    df = pd.DataFrame({
        "Medicine Name": medicine_list,
        "Type": ["Tablet","Tablet","Tablet","Syrup","Tablet","Tablet","Tablet",
                 "Tablet","Tablet","Tablet","Tablet","Cream","Tablet","Syrup","Syrup",
                 "Tablet","Suspension","Tablet","Tablet","Tablet","Capsule","Tablet",
                 "Tablet","Tablet","Tablet","Tablet","Eye Drop","Tablet","Syrup","Syrup"],
        "Dose": [""]*30,
        "Stock": [10]*30,
        "Start Date": [datetime.now().strftime("%Y-%m-%d")]*30,
        "End Date": [(datetime.now() + timedelta(days=30)).strftime("%Y-%m-%d")]*30
    })
    df.to_csv(CSV_PATH, index=False)

# Load medicine CSV
df = pd.read_csv(CSV_PATH)
print("📋 Medicine Data Loaded:")
print(df.head())


📋 Medicine Data Loaded:
   ID             Medicine Name  \
0   1   Avastin 400mg Injection   
1   2  Augmentin 625 Duo Tablet   
2   3       Azithral 500 Tablet   
3   4          Ascoril LS Syrup   
4   5         Aciloc 150 Tablet   

                                         Composition  \
0                                Bevacizumab (400mg)   
1    Amoxycillin  (500mg) +  Clavulanic Acid (125mg)   
2                               Azithromycin (500mg)   
3  Ambroxol (30mg/5ml) + Levosalbutamol (1mg/5ml)...   
4                                 Ranitidine (150mg)   

                                                Uses  \
0   Cancer of colon and rectum Non-small cell lun...   
1                  Treatment of Bacterial infections   
2                  Treatment of Bacterial infections   
3                      Treatment of Cough with mucus   
4  Treatment of Gastroesophageal reflux disease (...   

                                        Side_effects  \
0  Rectal bleeding Taste change Hea

In [16]:
# ---------------- Step 2: Stock Check ----------------
def check_stock(data):
    data["Stock"] = pd.to_numeric(data["Stock"], errors='coerce')
    for _, row in data.iterrows():
        med_name = row["Medicine Name"]
        stock = row["Stock"]
        if stock < 5:
            print(f"⚠️ Low stock: {med_name} → Only {stock} left")
        else:
            print(f"✅ Stock OK: {med_name} → {stock} left")

In [17]:
# ---------------- Step 3: Record Medicine Taken ----------------
def record_medicine_taken(medicine_name, notes=""):
    now = datetime.now()
    taken_record = {
        "Medicine Name": medicine_name,
        "Date": now.strftime("%Y-%m-%d"),
        "Time": now.strftime("%H:%M"),
        "Notes": notes
    }

    # Append to CSV
    try:
        taken_df = pd.read_csv("Medicine_Taken.csv")
        taken_df = pd.concat([taken_df, pd.DataFrame([taken_record])], ignore_index=True)
    except FileNotFoundError:
        taken_df = pd.DataFrame([taken_record])

    taken_df.to_csv("Medicine_Taken.csv", index=False)
    print(f"✅ Recorded: {medicine_name} at {taken_record['Time']} on {taken_record['Date']}")
    if notes:
        print(f"📝 Notes: {notes}")


In [18]:
# ---------------- Step 4: Take Medicine & Update Stock ----------------
def take_medicine(data, medicine_name, notes=""):
    if medicine_name not in data["Medicine Name"].values:
        print(f"❌ Medicine '{medicine_name}' not in your list!")
        return

    row = data[data["Medicine Name"] == medicine_name].iloc[0]
    start_date = pd.to_datetime(row["Start_Date"], dayfirst=True).date()
    end_date = pd.to_datetime(row["End_Date"], dayfirst=True).date()
    today = datetime.now().date()

    if not (start_date <= today <= end_date):
        print(f"❌ Medicine '{medicine_name}' not scheduled for today!")
        return

    record_medicine_taken(medicine_name, notes)
    data.loc[data["Medicine Name"] == medicine_name, "Stock"] -= 1
    data.to_csv(CSV_PATH, index=False)

    stock = data.loc[data["Medicine Name"] == medicine_name, "Stock"].values[0]
    if stock < 5:
        print(f"⚠️ Low stock warning: {medicine_name} → Only {stock} left")
    else:
        print(f"✅ Stock updated: {medicine_name} → {stock} left")


In [19]:

# ---------------- Step 5: Reports / History ----------------
def show_report(start_date=None, end_date=None):
    try:
        taken_df = pd.read_csv("Medicine_Taken.csv")
    except FileNotFoundError:
        print("❌ No medicine taken records found!")
        return

    taken_df["Date"] = pd.to_datetime(taken_df["Date"])
    if start_date:
        start_date = pd.to_datetime(start_date)
    else:
        start_date = taken_df["Date"].min()
    if end_date:
        end_date = pd.to_datetime(end_date)
    else:
        end_date = taken_df["Date"].max()

    filtered = taken_df[(taken_df["Date"] >= start_date) & (taken_df["Date"] <= end_date)]

    print(f"\n📋 Medicines taken from {start_date.date()} to {end_date.date()}:")
    if filtered.empty:
        print("No records found in this period.")
    else:
        print(filtered.to_string(index=False))

# ---------------- Example Usage ----------------
check_stock(df)  # Step 2: Check stock
take_medicine(df, "Alex Syrup", notes="After breakfast")  # Step 4: Take medicine
show_report("2025-09-20", "2025-09-27")  # Step 5: Report

✅ Stock OK: Avastin 400mg Injection → 11 left
✅ Stock OK: Augmentin 625 Duo Tablet → 24 left
✅ Stock OK: Azithral 500 Tablet → 19 left
✅ Stock OK: Ascoril LS Syrup → 15 left
✅ Stock OK: Aciloc 150 Tablet → 12 left
✅ Stock OK: Allegra 120mg Tablet → 25 left
✅ Stock OK: Avil 25 Tablet → 11 left
✅ Stock OK: Aricep 5 Tablet → 30 left
✅ Stock OK: Amoxyclav 625 Tablet → 23 left
✅ Stock OK: Atarax 25mg Tablet → 27 left
✅ Stock OK: Azee 500 Tablet → 15 left
✅ Stock OK: Anovate Cream → 15 left
✅ Stock OK: Allegra-M Tablet → 28 left
✅ Stock OK: Ascoril D Plus Syrup Sugar Free → 25 left
✅ Stock OK: Alex Syrup → 8 left
✅ Stock OK: Armotraz Tablet → 12 left
✅ Stock OK: Augmentin Duo Oral Suspension → 28 left
✅ Stock OK: Albendazole 400mg Tablet → 7 left
✅ Stock OK: Arkamin Tablet → 26 left
✅ Stock OK: Allegra 180mg Tablet → 25 left
✅ Stock OK: Altraday Capsule SR → 6 left
✅ Stock OK: Atarax 10mg Tablet → 28 left
✅ Stock OK: Aldigesic-SP Tablet → 16 left
✅ Stock OK: Aldactone Tablet → 10 left
✅ Stoc

In [25]:
# Step:6.........Predict stock running out
import pandas as pd
from datetime import datetime

# Load medicine taken records
taken_df = pd.read_csv("Medicine.csv")
taken_df["Date"] = pd.to_datetime(taken_df["Start_Date"])

# Example: predict for a specific medicine
medicine_name = "Alex Syrup"
medicine_history = taken_df[taken_df["Medicine Name"] == medicine_name]

if medicine_history.empty:
    print(f"No data to predict for {medicine_name}")
else:
    # Count doses per day
    daily_count = medicine_history.groupby("Date").size()
    avg_daily_consumption = daily_count.mean()

    # Get current stock
    current_stock = df.loc[df["Medicine Name"] == medicine_name, "Stock"].values[0]

    # Predict days until stock runs out
    if avg_daily_consumption > 0:
        days_left = current_stock / avg_daily_consumption
        print(f"📊 Prediction: {medicine_name} will run out in ~{days_left:.1f} days")
    else:
        print(f"{medicine_name} has not been taken yet, cannot predict stock depletion")


📊 Prediction: Alex Syrup will run out in ~8.0 days


/tmp/ipython-input-781644465.py:7: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  taken_df["Date"] = pd.to_datetime(taken_df["Start_Date"])


In [20]:
# Step: Get all medicine info for a specific patient
patient_name = "Imran Ali"

# Filter rows for this patient
patient_data = df[df["Patient Name"] == patient_name]

# Display information
if patient_data.empty:
    print(f"❌ No records found for patient '{patient_name}'")
else:
    print(f"📋 Medicine info for {patient_name}:")
    print(patient_data.to_string(index=False))

📋 Medicine info for Imran Ali:
 ID   Medicine Name          Composition                              Uses                   Side_effects                                                                                                                     Image URL Manufacturer  Excellent Review %  Average Review %  Poor Review %                 Dosage  Frequency Start_Date   End_Date Time_of_Day Status Patient Name              Disease  Stock
 11 Azee 500 Tablet Azithromycin (500mg) Treatment of Bacterial infections Nausea Abdominal pain Diarrhea https://onemg.gumlet.io/l_watermark_346,w_480,h_480/a_ignore,w_480,h_480,c_fit,q_auto,f_auto/cropped/jbpnjvst3ph0xnrq199o.jpg    Cipla Ltd                  37                38             25 2 Tablets (250mg each) Once daily 21/09/2025 26/09/2025       Night Missed    Imran Ali Bacterial infections     15


In [21]:
# Step: Get patient info from medicine name
medicine_name = "Alex Syrup"  # Replace with the medicine you know

# Filter the data for this medicine
med_data = df[df["Medicine Name"] == medicine_name]

if med_data.empty:
    print(f"❌ No records found for medicine '{medicine_name}'")
else:
    # Show only patient name, start date, end date, and stock
    print(med_data[["Patient Name", "Start_Date", "End_Date", "Stock"]].to_string(index=False))


  Patient Name Start_Date   End_Date  Stock
Kashif Mehmood 21/09/2025 26/09/2025      8
